In [1]:
import numpy as np
import shapely
from czml3 import CZML_VERSION, Document, Packet
from czml3.properties import (
    Color,
    Ellipsoid,
    EllipsoidRadii,
    Material,
    Polygon,
    Polyline,
    PolylineMaterial,
    PositionList,
    SolidColorMaterial,
)

import czml3_ext
from czml3_ext import packets
from czml3_ext.colours import (
    RGBA_blue,
    RGBA_orange,
    RGBA_white,
    create_palette,
)
from czml3_ext.helpers import get_border

In [ ]:
print(f"Using czml3-ext version: {czml3_ext.__version__}")

## Border
A single border

In [3]:
border = packets.border("Israel")

Multiple borders

In [4]:
borders = packets.border(["syria", "lebanon", "jordan"])

## Sensors
A single sensor

In [5]:
sensor1 = packets.sensor(
    np.array([[31.4], [34.7], [1000]]),
    10,
    30,
    100,
    20,
    10_000,
    5000,
    name="A Sensor",  # kwargs add to the CZML3 packet
    ellipsoid=Ellipsoid(
        radii=EllipsoidRadii(
            cartesian=[0, 0, 0]
        ),  # required to create an Ellipsoid(), and is ignored by czml3-ext
        material=Material(
            solidColor=SolidColorMaterial(
                color=Color(rgba=RGBA_blue.get_with_temp_alpha(100))
            )
        ),
        outlineColor=Color(rgba=RGBA_white),
        fill=True,
        outline=True,
    ),
)

In [6]:
sensor2 = packets.sensor(
    np.array([[31.4], [34.9], [1000]]),
    10,
    30,
    300,
    20,
    10_000,
    5000,
    name="A Large Sensor",  # kwargs add to the CZML3 packet
    ellipsoid=Ellipsoid(
        radii=EllipsoidRadii(
            cartesian=[0, 0, 0]
        ),  # required to create an Ellipsoid(), and is ignored by czml3-ext
        material=Material(
            solidColor=SolidColorMaterial(
                color=Color(rgba=RGBA_blue.get_with_temp_alpha(100))
            )
        ),
        fill=True,
    ),
)  # we could use slicePartitions=3 (the minimum), but this makes the fill wrong

In [7]:
sensor3 = packets.sensor(
    np.array([[31.4], [35.12], [1000]]),
    10,
    30,
    300,
    20,
    10_000,
    5000,
    name="A Confusing Ellipsoid Sensor",  # kwargs add to the CZML3 packet
    ellipsoid=Ellipsoid(
        radii=EllipsoidRadii(
            cartesian=[0, 0, 0]
        ),  # required to create an Ellipsoid(), and is ignored by czml3-ext
        material=Material(
            solidColor=SolidColorMaterial(
                color=Color(rgba=RGBA_blue.get_with_temp_alpha(100))
            )
        ),
        outlineColor=Color(rgba=RGBA_white),
        fill=True,
        outline=True,
    ),
    max_ellipsoid_angle=360,
)

In [8]:
sensor4 = packets.sensor(
    np.array([[31.4], [35.34], [1000]]),
    10,
    30,
    300,
    20,
    10_000,
    5000,
    name="A Bad Ellipsoid Sensor",  # kwargs add to the CZML3 packet
    ellipsoid=Ellipsoid(
        radii=EllipsoidRadii(
            cartesian=[0, 0, 0]
        ),  # required to create an Ellipsoid(), and is ignored by czml3-ext
        material=Material(
            solidColor=SolidColorMaterial(
                color=Color(rgba=RGBA_blue.get_with_temp_alpha(100))
            )
        ),
        outlineColor=Color(rgba=RGBA_white),
        fill=True,
        outline=True,
        slicePartitions=3,
    ),
    max_ellipsoid_angle=360,
)

Multiple sensors

In [9]:
ddm_LLA = np.array([[[31.75], [34.72], [0]], [[31.8], [34.68], [0]]])
sensors1 = packets.sensor(
    ddm_LLA,
    [50, 90],
    [60, 30],
    [10, 50],
    [20, 20],
    [10_000, 20_000],
    [5_000, 2_000],
    name="My sensor",  # a kwarg sequence assigns each property to their appropriate sensor
    polyline=Polyline(
        positions=PositionList(
            cartesian=[0, 0, 0]
        ),  # required to create a Polyline(), and is ignored by czml3-ext
        material=PolylineMaterial(
            solidColor=SolidColorMaterial(color=Color(rgba=RGBA_orange))
        ),
    ),  # a kwarg that is not a sequence will be applied to all sensors
    ellipsoid=Ellipsoid(
        radii=EllipsoidRadii(
            cartesian=[0, 0, 0]
        ),  # required to create an Ellipsoid(), and is ignored by czml3-ext
        material=Material(
            solidColor=SolidColorMaterial(
                color=Color(rgba=RGBA_blue.get_with_temp_alpha(100))
            )
        ),
        outlineColor=Color(rgba=RGBA_orange),
        fill=True,
        outline=True,
    ),
)

## Grid
Create a grid in Israel

In [10]:
ddm_LLA_israel = get_border("israel")
poly_shape = shapely.Polygon(ddm_LLA_israel[:, [0, 1], 0].tolist())

In [11]:
x = np.linspace(ddm_LLA_israel[:, 1, 0].min(), ddm_LLA_israel[:, 1, 0].max(), 20)
y = np.linspace(ddm_LLA_israel[:, 0, 0].min(), ddm_LLA_israel[:, 0, 0].max(), 20)
xv, yv = np.meshgrid(x, y)
ddm_points = []
for i in range(x.shape[0]):
    for j in range(y.shape[0]):
        if not poly_shape.contains(shapely.Point(yv[j, i], xv[j, i])):
            continue
        ddm_points.append([yv[j, i], xv[j, i], 0])
ddm_LLA_points = np.array(ddm_points).reshape((-1, 3, 1))

Define the colour palette

In [12]:
rgba = create_palette(
    [RGBA_blue, RGBA_white, RGBA_orange, RGBA_blue], ddm_LLA_points.shape[0]
)

Create the grid, and cut it using the border

In [13]:
grid = packets.grid(
    ddm_LLA_points,
    polygon=[
        Polygon(
            positions=PositionList(cartographicDegrees=[0, 0, 0]),
            material=Material(solidColor=SolidColorMaterial(color=Color(rgba=c))),
            outlineColor=Color(rgba=[255, 255, 255, 255]),
            outline=True,
        )
        for c in rgba
    ],
    name=[f"Square {i}" for i in range(len(rgba))],
    description=[
        f"Colour: [{c[0]:.2f}, {c[1]:.2f}, {c[2]:.2f}, {c[3]:.2f}]" for c in rgba
    ],
    ddm_LLA_cut=ddm_LLA_israel,
)

## Export
Export to file as you would any other `czml3.Document`.

In [14]:
doc = Document(
    packets=(
        [Packet(name="example", id="document", version=CZML_VERSION)]
        + border
        + borders
        + sensor1
        + sensor2
        + sensor3
        + sensor4
        + sensors1
        + grid
    )
)

In [15]:
with open("example.czml", "w") as f:
    f.write(doc.dumps())